In [1]:
import duckdb
conn = duckdb.connect()
import os
import pandas as pd

In [2]:
#archivo ventas

drive_base_path = 'datan/'
filename = 'sell-in.txt'
filepath = os.path.join(drive_base_path, filename)
sell = pd.read_csv(filepath, sep='\t')

In [3]:
resultado_producto=sell.groupby(['periodo','product_id'])['tn'].sum().reset_index()

In [4]:
resultado_producto

,periodo,product_id,tn
0,201701,20001,934.77222
1,201701,20002,550.15707
2,201701,20003,1063.45835
3,201701,20004,555.91614
4,201701,20005,494.27011
...,...,...,...
31238,201912,21265,0.05007
31239,201912,21266,0.05121
31240,201912,21267,0.01569
31241,201912,21271,0.00298


In [5]:
query = """
-- Transformación completa: campo clase, media móvil y lags
WITH base_with_clase AS (
    -- Primero creamos el campo clase (tn en periodo+2)
    SELECT 
        r1.periodo,
        r1.product_id,
        r1.tn,
        r2.tn AS clase
    FROM resultado_producto r1
    LEFT JOIN resultado_producto r2 
        ON r1.product_id = r2.product_id 
        AND r2.periodo = (
            CASE 
                WHEN r1.periodo % 100 = 11 THEN (r1.periodo + 90) -- Nov -> Ene siguiente año
                WHEN r1.periodo % 100 = 12 THEN (r1.periodo + 90) -- Dic -> Feb siguiente año  
                ELSE r1.periodo + 2 -- Resto de meses
            END
        )
),

-- Función para calcular el periodo anterior correctamente
periodo_anterior AS (
    SELECT *,
        CASE 
            WHEN periodo % 100 = 1 THEN periodo - 89  -- Ene -> Dic año anterior
            ELSE periodo - 1                          -- Resto de meses
        END AS periodo_prev
    FROM base_with_clase
),

-- Creamos los lags y la media móvil
final_result AS (
    SELECT 
        p.periodo,
        p.product_id,
        p.tn,
        p.clase,
        
        -- Lags del 1 al 11 (usando LAG window function)
        LAG(p.tn, 1) OVER (PARTITION BY p.product_id ORDER BY p.periodo) AS tn_1,
        LAG(p.tn, 2) OVER (PARTITION BY p.product_id ORDER BY p.periodo) AS tn_2,
        LAG(p.tn, 3) OVER (PARTITION BY p.product_id ORDER BY p.periodo) AS tn_3,
        LAG(p.tn, 4) OVER (PARTITION BY p.product_id ORDER BY p.periodo) AS tn_4,
        LAG(p.tn, 5) OVER (PARTITION BY p.product_id ORDER BY p.periodo) AS tn_5,
        LAG(p.tn, 6) OVER (PARTITION BY p.product_id ORDER BY p.periodo) AS tn_6,
        LAG(p.tn, 7) OVER (PARTITION BY p.product_id ORDER BY p.periodo) AS tn_7,
        LAG(p.tn, 8) OVER (PARTITION BY p.product_id ORDER BY p.periodo) AS tn_8,
        LAG(p.tn, 9) OVER (PARTITION BY p.product_id ORDER BY p.periodo) AS tn_9,
        LAG(p.tn, 10) OVER (PARTITION BY p.product_id ORDER BY p.periodo) AS tn_10,
        LAG(p.tn, 11) OVER (PARTITION BY p.product_id ORDER BY p.periodo) AS tn_11,
        
        -- Media móvil de los últimos 12 meses (incluyendo el actual)
        -- Solo calcula cuando hay al menos 1 valor no nulo
        CASE 
            WHEN COUNT(p.tn) OVER (
                PARTITION BY p.product_id 
                ORDER BY p.periodo 
                ROWS BETWEEN 11 PRECEDING AND CURRENT ROW
            ) > 0
            THEN AVG(p.tn) OVER (
                PARTITION BY p.product_id 
                ORDER BY p.periodo 
                ROWS BETWEEN 11 PRECEDING AND CURRENT ROW
            )
            ELSE NULL
        END AS media_movil_12m,
        
        -- Cantidad de meses con datos para la media móvil
        COUNT(p.tn) OVER (
            PARTITION BY p.product_id 
            ORDER BY p.periodo 
            ROWS BETWEEN 11 PRECEDING AND CURRENT ROW
        ) AS meses_con_datos
        
    FROM periodo_anterior p
)

SELECT 
    periodo,
    product_id,
    tn,
    clase,
    tn_1, tn_2, tn_3, tn_4, tn_5, tn_6, 
    tn_7, tn_8, tn_9, tn_10, tn_11,
    media_movil_12m,
    meses_con_datos
FROM final_result
ORDER BY periodo, product_id;

"""

In [6]:
duckdb.register('resultado_producto', resultado_producto)
resultado_prod = duckdb.query(query).to_df()

In [7]:
conn.close()

In [12]:
query = """
SELECT * 
FROM productos 
WHERE productos.periodo = 201812
  AND productos.product_id IN (20002, 20003, 20006, 20010, 20011, 20018, 20019, 20021,
                               20026, 20028, 20035, 20039, 20042, 20044, 20045, 20046, 20049,
                               20051, 20052, 20053, 20055, 20008, 20001, 20017, 20086, 20180,
                               20193, 20320, 20532, 20612, 20637, 20807, 20838)
ORDER BY productos.product_id;
"""

vamos a interntar predecir diciembre 19 por eso cambiamos original 201812


In [13]:
duckdb.register('productos', resultado_prod)
resultado_prod2 = duckdb.query(query).to_df()

In [14]:
df_clean = resultado_prod2.drop(['periodo', 'product_id','media_movil_12m','meses_con_datos'], axis=1)

In [15]:
df_clean

,tn,clase,tn_1,tn_2,tn_3,tn_4,tn_5,tn_6,tn_7,tn_8,tn_9,tn_10,tn_11
0,1486.68669,1259.09363,1813.01511,2295.19832,1438.67455,1800.96168,1470.41009,1150.79169,1293.89788,1251.28462,1856.83534,1043.76470,1169.07532
1,1009.45458,1043.01349,1766.81068,1378.49032,954.23575,1161.88430,977.40239,1033.82845,1103.39191,999.20934,966.86044,712.00087,984.80167
2,769.82869,758.32657,1206.91773,1313.34211,912.34156,955.97079,656.22700,660.73323,784.35885,765.47838,778.55594,788.30749,907.56304
3,407.75925,479.99914,566.66809,513.15472,478.04388,615.70617,515.20419,468.15260,865.28861,748.44391,862.19361,588.56272,470.33785
4,426.32899,476.98787,433.50170,532.45644,436.96269,554.82147,526.38149,554.57063,707.59267,691.53246,765.98901,506.25385,469.29224
5,285.02947,337.76009,414.97753,612.50721,480.60235,582.83104,331.96807,223.87746,227.24082,171.74107,653.77607,477.48363,298.25586
6,321.09714,431.62938,289.13976,177.75576,189.59850,191.07270,300.26178,437.75550,484.04538,562.70214,526.99374,601.26066,340.75314
7,259.32724,308.71060,286.83676,331.23254,288.35292,374.95908,351.60065,316.45841,533.53335,550.29417,488.79258,377.84497,291.70926
8,326.01506,265.84135,371.52958,161.58557,282.43485,375.61778,325.03223,420.33781,388.43687,543.06908,510.33171,337.54792,342.16945
9,446.69747,323.66178,532.98143,552.71975,417.95455,387.73155,351.05610,262.33076,356.42982,290.39581,321.26878,629.89543,243.71984


In [16]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np

# ========================================
# PASO 1: CARGAR DATASET DE ENTRENAMIENTO
# ========================================

# Cargar el dataset de entrenamiento (33 registros estratégicos de 201812)
df_train = df_clean  # Ajustar ruta según corresponda

print("Dataset de Entrenamiento:")
print(f"Filas: {len(df_train)}")
print(f"Columnas: {list(df_train.columns)}")
print("\nPrimeros registros:")
print(df_train.head())

# ========================================
# PASO 2: PREPARAR VARIABLES
# ========================================

# Variables independientes (features): 12 campos de toneladas
X = df_train[['tn', 'tn_1', 'tn_2', 'tn_3', 'tn_4', 'tn_5', 'tn_6', 
              'tn_7', 'tn_8', 'tn_9', 'tn_10', 'tn_11']]

# Variable dependiente (target): toneladas del mes+2 (201902)
y = df_train['clase']

print(f"\nVariables independientes (X): {X.shape}")
print(f"Variable dependiente (y): {y.shape}")

# Verificar que no hay valores nulos
print(f"\nValores nulos en X: {X.isnull().sum().sum()}")
print(f"Valores nulos en y: {y.isnull().sum()}")

# ========================================
# PASO 3: ENTRENAR REGRESIÓN LINEAL SIMPLE
# ========================================

modelo = LinearRegression()

# Entrenar el modelo
modelo.fit(X, y)

print("\n" + "="*50)
print("MODELO ENTRENADO EXITOSAMENTE")
print("="*50)

# ========================================
# PASO 4: EVALUAR EL MODELO
# ========================================

# Hacer predicciones en el mismo conjunto de entrenamiento
y_pred = modelo.predict(X)

# Calcular métricas
mse = mean_squared_error(y, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y, y_pred)
r2 = r2_score(y, y_pred)

print(f"\nMÉTRICAS DEL MODELO:")
print(f"R² Score: {r2:.4f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")

# ========================================
# PASO 5: COEFICIENTES DEL MODELO
# ========================================

print(f"\nCOEFICIENTES DE LA REGRESIÓN:")
print(f"Intercepto: {modelo.intercept_:.4f}")

# Mostrar coeficientes para cada variable
coeficientes = pd.DataFrame({
    'Variable': X.columns,
    'Coeficiente': modelo.coef_
})
coeficientes['Abs_Coeficiente'] = np.abs(coeficientes['Coeficiente'])
coeficientes = coeficientes.sort_values('Abs_Coeficiente', ascending=False)

print("\nCoeficientes ordenados por importancia:")
for _, row in coeficientes.iterrows():
    print(f"{row['Variable']:>6}: {row['Coeficiente']:>10.4f}")

# ========================================
# PASO 6: ECUACIÓN DE LA REGRESIÓN
# ========================================

print(f"\nECUACIÓN DE LA REGRESIÓN:")
ecuacion = f"clase = {modelo.intercept_:.4f}"
for i, coef in enumerate(modelo.coef_):
    variable = X.columns[i]
    signo = "+" if coef >= 0 else ""
    ecuacion += f" {signo}{coef:.4f}*{variable}"

print(ecuacion)

# ========================================
# PASO 7: ANÁLISIS DE RESIDUOS
# ========================================

residuos = y - y_pred
print(f"\nANÁLISIS DE RESIDUOS:")
print(f"Media de residuos: {np.mean(residuos):.6f}")
print(f"Std de residuos: {np.std(residuos):.4f}")
print(f"Min residuo: {np.min(residuos):.4f}")
print(f"Max residuo: {np.max(residuos):.4f}")

# ========================================
# PASO 8: GUARDAR MODELO Y RESULTADOS
# ========================================

# Crear DataFrame con resultados
resultados = pd.DataFrame({
    'Real': y,
    'Prediccion': y_pred,
    'Residuo': residuos,
    'Residuo_Abs': np.abs(residuos)
})

print(f"\nRESULTADOS DETALLADOS:")
print(resultados.round(4))

# Guardar resultados
resultados.to_csv('resultados_entrenamiento.csv', index=False)
coeficientes.to_csv('coeficientes_modelo.csv', index=False)

print(f"\n✅ ENTRENAMIENTO COMPLETADO")
print(f"✅ Archivos guardados: resultados_entrenamiento.csv, coeficientes_modelo.csv")
print(f"\nEl modelo está listo para hacer predicciones con nuevos datos.")

Dataset de Entrenamiento:
Filas: 33
Columnas: ['tn', 'clase', 'tn_1', 'tn_2', 'tn_3', 'tn_4', 'tn_5', 'tn_6', 'tn_7', 'tn_8', 'tn_9', 'tn_10', 'tn_11']

Primeros registros:
           tn       clase        tn_1        tn_2        tn_3        tn_4  \
0  1486.68669  1259.09363  1813.01511  2295.19832  1438.67455  1800.96168   
1  1009.45458  1043.01349  1766.81068  1378.49032   954.23575  1161.88430   
2   769.82869   758.32657  1206.91773  1313.34211   912.34156   955.97079   
3   407.75925   479.99914   566.66809   513.15472   478.04388   615.70617   
4   426.32899   476.98787   433.50170   532.45644   436.96269   554.82147   

         tn_5        tn_6        tn_7        tn_8        tn_9       tn_10  \
0  1470.41009  1150.79169  1293.89788  1251.28462  1856.83534  1043.76470   
1   977.40239  1033.82845  1103.39191   999.20934   966.86044   712.00087   
2   656.22700   660.73323   784.35885   765.47838   778.55594   788.30749   
3   515.20419   468.15260   865.28861   748.44391   862.

In [17]:
drive_base_path = 'datan/'
filename = 'productos_a_predecir.txt'
filepath = os.path.join(drive_base_path, filename)
p780 = pd.read_csv(filepath)

In [18]:
query = """
SELECT * 
FROM productos 
WHERE productos.periodo = 201912
ORDER BY productos.product_id;
"""

In [19]:
duckdb.register('resultado_producto', resultado_prod)
resultado_apredecir = duckdb.query(query).to_df()

In [20]:
resultados_filtrados = resultado_apredecir[resultado_apredecir['product_id'].isin(p780['product_id'])]

In [21]:
query = """
SELECT * 
FROM productos 
WHERE productos.meses_con_datos = 12
ORDER BY productos.product_id;
"""

In [22]:
duckdb.register('productos', resultados_filtrados)
resultado_12m = duckdb.query(query).to_df()

In [23]:
X = resultado_12m[['tn', 'tn_1', 'tn_2', 'tn_3', 'tn_4', 'tn_5', 'tn_6', 
              'tn_7', 'tn_8', 'tn_9', 'tn_10', 'tn_11']]
y_pred = modelo.predict(X)

In [24]:
prediccion = pd.DataFrame({
    'product_id': resultado_12m['product_id'],  # Tomar del DataFrame existente
    'tn': y_pred                        # Array de predicciones
})

In [25]:
# Encontrar product_id que están en resultados_filtrados pero NO en prediccion
product_ids_faltantes = resultados_filtrados[
    ~resultados_filtrados['product_id'].isin(prediccion['product_id'])
]['product_id'].tolist()

In [26]:
# Extraer las medias de 12 meses para los product_id faltantes
medias_faltantes = resultados_filtrados[
    resultados_filtrados['product_id'].isin(product_ids_faltantes)
][['product_id', 'media_movil_12m']].copy()

In [27]:
predicciones_faltantes = pd.DataFrame({
    'product_id': medias_faltantes['product_id'],
    'tn': medias_faltantes['media_movil_12m']
})

In [28]:
prediccion_completa = pd.concat([
    prediccion,                    # Predicciones del modelo
    predicciones_faltantes         # Predicciones con media_12meses
], ignore_index=True)

In [29]:
prediccion_completa

,product_id,tn
0,20001,1162.707525
1,20002,1183.640604
2,20003,684.763931
3,20004,580.484961
4,20005,563.560780
...,...,...
775,21252,0.178011
776,21265,0.089541
777,21266,0.094659
778,21267,0.092835


In [ ]:
prediccion_completa.to_csv('regresion_pred_201912.csv',index=False)

In [ ]:
resultado_diciembre=resultado_producto[resultado_producto['periodo'] == 201912]

In [ ]:
resultado_diciembre

In [ ]:
prediccion_con_real = prediccion_completa.merge(
   resultado_diciembre[['product_id', 'tn']], 
   on='product_id', 
   how='left'
).rename(columns={'tn_x': 'pred', 'tn_y': 'real'})

In [ ]:
prediccion_con_real

In [ ]:
# Calcular Total Forecast Error con tu dataframe
total_forecast_error = np.sum(np.abs(prediccion_con_real['real'] - prediccion_con_real['pred'])) / np.sum(prediccion_con_real['real']) * 100

print(f"Total Forecast Error: {total_forecast_error:.2f}%")

# Verificar datos válidos
print(f"Productos analizados: {len(prediccion_con_real)}")
print(f"Productos con datos reales: {prediccion_con_real['real'].notna().sum()}")